# Column analysis

This notebook is dedicated to column analysis prior to data preparation

In [1]:
from pyspark.sql import SparkSession

# Add here your team number teamx
team = 14

# location of your Hive database in HDFS
warehouse = "project/hive/warehouse"

spark = SparkSession.builder\
        .appName("Team {} - spark Column Analysis".format(team))\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 15:04:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/22 15:04:12 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/04/22 15:04:12 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
spark.sql("SHOW DATABASES;").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|             retake1|
|             root_db|
|                show|
|     team0_projectdb|
|    team11_projectdb|
|team12_hive_proje...|
|    team13_projectdb|
|    team14_projectdb|
|    team15_projectdb|
|    team16_projectdb|
|    team17_projectdb|
|    team18_projectdb|
|    team19_projectdb|
|     team1_projectdb|
|    team20_projectdb|
|    team21_projectdb|
|    team22_projectbd|
|    team22_projectdb|
|    team23_projectdb|
+--------------------+
only showing top 20 rows



In [3]:
db = 'team14_projectdb'

In [4]:
print(spark.catalog.listTables(db))

[Table(name='job_descriptions_part', catalog='spark_catalog', namespace=['team14_projectdb'], description=None, tableType='EXTERNAL', isTemporary=False)]


In [5]:
table_name = 'job_descriptions_part'
df = spark.read.format("avro").table(f'{db}.{table_name}')
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- job_id: long (nullable = true)
 |-- experience: string (nullable = true)
 |-- qualifications: string (nullable = true)
 |-- salary_range: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: decimal(9,6) (nullable = true)
 |-- longitude: decimal(9,6) (nullable = true)
 |-- company_size: integer (nullable = true)
 |-- job_posting_date: date (nullable = true)
 |-- contact_person: string (nullable = true)
 |-- preference: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- role: string (nullable = true)
 |-- job_portal: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- benefits: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- responsibilities: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- company_profile: string (nullable = true)
 |-- work_type: str

In [6]:
df = df.na.drop()

In [7]:
first_row_dict = df.limit(1).toPandas().iloc[0].to_dict()
print(first_row_dict)

25/04/22 15:04:34 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


{'id': 1173529, 'job_id': 364241521459786, 'experience': '4 to 11 Years', 'qualifications': 'BA', 'salary_range': '$62K-$118K', 'location': 'Antananarivo', 'country': 'Madagascar', 'latitude': Decimal('-18.879200'), 'longitude': Decimal('46.845100'), 'company_size': 121206, 'job_posting_date': datetime.date(2022, 1, 31), 'contact_person': 'Calvin Thompson', 'preference': 'Female', 'contact': '472.993.7608x09238', 'job_title': 'Project Coordinator', 'role': 'Construction Project Coordinator', 'job_portal': 'Jobs2Careers', 'job_description': 'Construction Project Coordinators assist in managing construction projects, handling documentation, budget tracking, and coordinating subcontractors.', 'benefits': "{'Casual Dress Code, Social and Recreational Activities, Employee Referral Programs, Health and Wellness Facilities, Life and Disability Insurance'}", 'skills': 'Construction project management Building codes and regulations knowledge Budgeting and cost control Construction scheduling Co

In [9]:
from pyspark.sql.functions import col, countDistinct, udf, from_json, when, regexp_replace
from pyspark.sql.types import StringType, StructType, StructField
import json

categorical = ['qualifications', 'location', 'country', 'work_type', 'preference', 'job_title', 'role', 'job_portal', 'company_name']

for column in categorical:
    count = df.select(countDistinct(col(column))).collect()[0][0]
    print(f"Column: {column}")
    print(count)
    print("-" * 50)

Column: qualifications
10
--------------------------------------------------


Column: location
214
--------------------------------------------------


Column: country
216
--------------------------------------------------


Column: work_type
5
--------------------------------------------------


Column: preference
3
--------------------------------------------------


Column: job_title
147
--------------------------------------------------


Column: role
376
--------------------------------------------------


Column: job_portal
16
--------------------------------------------------


Column: company_name
888
--------------------------------------------------


In [10]:
def is_valid_json(s):
    try:
        json.loads(s)
        return True
    except:
        return False

is_valid_json_udf = udf(is_valid_json)

# Add column indicating if JSON is valid
df = df.withColumn("is_valid_json", is_valid_json_udf(col("company_profile")))

# Count valid and invalid JSON rows
valid_json_count = df.filter(col("is_valid_json") == True).count()
invalid_json_count = df.filter(col("is_valid_json") == False).count()

print(f"Valid Company Profile JSONs: {valid_json_count}")
print(f"Invalid Company Profile JSONs: {invalid_json_count}")


Valid Company Profile JSONs: 1608618
Invalid Company Profile JSONs: 7322


In [11]:
df_valid = df.filter(col("is_valid_json") == True)

In [12]:
df = df.withColumn("Company_Profile_Cleaned", regexp_replace(col("company_profile"), "'", '"'))

company_profile_schema = StructType([
    StructField("Sector", StringType(), True),
    StructField("Industry", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Zip", StringType(), True),
    StructField("Website", StringType(), True),
    StructField("Ticker", StringType(), True),
    StructField("CEO", StringType(), True)
])

df_valid = df.withColumn(
    "CompanyProfileParsed",
    from_json(col("Company_Profile_Cleaned"), company_profile_schema)
)

company_profile_df = df_valid.select("CompanyProfileParsed.*")
company_profile_df.show(5, truncate=False)

+-----------------------------+----------------------------------------+------------+--------+-----+----------------------------------+------+-------------------+
|Sector                       |Industry                                |City        |State   |Zip  |Website                           |Ticker|CEO                |
+-----------------------------+----------------------------------------+------------+--------+-----+----------------------------------+------+-------------------+
|Food and Beverage            |Food Manufacturing                      |Chicago     |IL      |60601|https://www.kraftheinzcompany.com/|KHC   |Miguel Patricio    |
|Financial Services           |Insurance: Property and Casualty (Stock)|Jacksonville|Florida |32204|www.fnf.com                       |FNF   |Mike Nolan         |
|Transportation/Infrastructure|Transportation/Infrastructure           |Melbourne   |VIC     |3000 |https://www.transurban.com/       |TCL   |Scott Charlton     |
|Automotive           

In [13]:
for column in company_profile_df.columns:
    count = company_profile_df.select(countDistinct(col(column))).collect()[0][0]
    print(f"Column: {column}")
    print(f"Unique values: {count}")
    print("-" * 50)

Column: Sector
Unique values: 204
--------------------------------------------------


Column: Industry
Unique values: 203
--------------------------------------------------


Column: City
Unique values: 341
--------------------------------------------------


Column: State
Unique values: 98
--------------------------------------------------


Column: Zip
Unique values: 491
--------------------------------------------------


Column: Website
Unique values: 869
--------------------------------------------------


Column: Ticker
Unique values: 807
--------------------------------------------------


Column: CEO
Unique values: 825
--------------------------------------------------


In [44]:
import gender_guesser.detector as gender
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

def get_ceo_gender(name):
    if name is None or name.strip() == "":
        return "unknown"
    first_name = name.split()[0]
    d = gender.Detector()
    result = d.get_gender(first_name)
    if result in ["male", "mostly_male"]:
        return "male"
    elif result in ["female", "mostly_female"]:
        return "female"
    else:
        return "unknown"

get_ceo_gender_udf = udf(get_ceo_gender, StringType())

company_profile_df = company_profile_df.withColumn("CEO_Gender", get_ceo_gender_udf(col("CEO")))

gender_counts = company_profile_df.groupBy("CEO_Gender").count()

print("Gender counts for CEOs:")
gender_counts.show()


ModuleNotFoundError: No module named 'gender_guesser'